In [3]:
# Loading Elior's model
import os
cuda = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = cuda

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForQuestionAnswering

root_dir = '/shared/lyuqing/probing_for_event/'
os.chdir(root_dir)


In [4]:
model_path = 'output_model_dir/MNLI_s_roberta'
te_model = AutoModelForSequenceClassification.from_pretrained(model_path).to('cuda:0')
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [3]:
model_path = 'output_model_dir/qamr_bert'
qa_model = AutoModelForQuestionAnswering.from_pretrained(model_path).to('cuda:0')
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [5]:
text = r"""
耶稣出生于公元1世纪。
"""

questions = [
    "Who is born?",
    "When is someone born?"
]

for question in questions:
    inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt").to('cuda:0')
    input_ids = inputs["input_ids"].tolist()[0]

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer_start_scores, answer_end_scores = qa_model(**inputs)

    answer_start = torch.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score
    
    print(text_tokens)
    print(answer_start, answer_end)
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

['[CLS]', 'who', 'is', 'born', '?', '[SEP]', '耶', '稣', '出', '生', '于', '公', '元', '1', '世', '纪', '。', '[SEP]']
tensor(0, device='cuda:0') tensor(1, device='cuda:0')
Question: Who is born?
Answer: [CLS]

['[CLS]', 'when', 'is', 'someone', 'born', '?', '[SEP]', '耶', '稣', '出', '生', '于', '公', '元', '1', '世', '纪', '。', '[SEP]']
tensor(12, device='cuda:0') tensor(17, device='cuda:0')
Question: When is someone born?
Answer: 公 元 1 世 纪



In [ ]:
def answer_ex_original(question, context):
    """Answers an extractive question. (without mapping gold tokens to bert tokens)"""
    
    if context and len(context) > 1:  # Capitalize the first letter of the context
        context = context[0].upper() + context[1:]
    question = question[0].upper() + question[1:]
    if question[-1] != '?':
        question = question + '?'
    
    input_tensor = ex_tokenizer(question, context, add_special_tokens=True, return_tensors="pt").to(
        'cuda:0')
    input_ids = input_tensor["input_ids"].tolist()[0]
    text_tokens = ex_tokenizer.convert_ids_to_tokens(input_ids)
    question_end_s = text_tokens.index('[SEP]') 


#     context_start = question_end_s + 2 # the starting token of the context
    context_start = question_end_s + 1 # the starting token of the context
    
    context_end = len(text_tokens) - 1 # the ending token of the context
    context_tokens = text_tokens[context_start:context_end]
    print(type(input_tensor))
    print(input_tensor)
    outputs = ex_qa_model(**input_tensor)
    answer_start_scores = outputs[0]
    answer_end_scores = outputs[1]
    start_probs = torch.softmax(answer_start_scores, dim=1).tolist()[0]
    end_probs = torch.softmax(answer_end_scores, dim=1).tolist()[0]
    print(start_probs)
    print(end_probs)

    answer_start = torch.argmax(answer_start_scores).tolist()  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores).tolist() + 1  # Get the most likely end of answer with the argmax of the score
    confidence = np.mean([start_probs[answer_start], end_probs[answer_end-1]])
    answer = ex_tokenizer.convert_tokens_to_string(ex_tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    span = (answer_start, answer_end)
    print(span, confidence)
    if span == (0,1): # answer = '<s>'
        span = None
        answer = None
    return {'span':span,
            'answer':answer, 
            'confidence':confidence,
            'bert_tokens':text_tokens
           }

In [ ]:
model_path = "output_model_dir/elior_bert-lc_mnli/"
te_model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=3).to('cuda:0')
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [3]:
def entailment(premise, hypothesis, premise_upper=True):
    """Compute the probability that the premise entails the hypothesis."""

    x = tokenizer.encode(premise, hypothesis, return_tensors='pt', truncation='only_first', max_length=tokenizer.max_len).to('cuda:'+str(0))
    logits = te_model(x)[0]

    entail_contradiction_logits = logits
    entail_idx = 2
    probs = entail_contradiction_logits.softmax(1)
    print(probs)

    entail_prob = float(probs[:, entail_idx])

    return entail_prob

In [6]:
premise = "I can run"
hypothesis = "I can run"
entailment(premise, hypothesis)

tensor([[0.0061, 0.6793, 0.3145]], device='cuda:0', grad_fn=<SoftmaxBackward>)


0.3145211935043335